In [1]:
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd 
import itertools
sns.set()
plt.style.use('ggplot')


r = requests.get("https://cropapptest.azurewebsites.net/queries/harvests")
df = pd.DataFrame.from_dict(r.json())
df['event_time'] = pd.to_datetime(df['event_time'])
df.head()

,aisle,batch_id,column,crop_yield,event_time,growapp_id,id,name,number_of_trays,over_production,shelf,tray_size,waste_defect,waste_disease,zone
0,E,1,11,6660.0,2022-07-15 12:45:46.343769,f28279e0-d0fd-463f-8455-aeca00d8400f,1,sunflower (micro),5,0.0,4,3.0,0.0,0.0,Farm
1,E,3,11,6004.0,2022-07-15 12:46:07.728404,fa35712f-09ff-4fca-9075-aeca00df2c22,2,sunflower (micro),5,0.0,1,3.0,0.0,0.0,Farm
2,E,4,1,100.0,2022-07-04 11:22:58.712672,d1552a36-e305-4389-8ada-aec200b06062,3,peashoots (babyleaf),5,0.0,4,3.0,0.0,0.0,Farm
3,E,5,1,100.0,2022-07-04 11:23:22.473038,b94ba47a-7e18-4e65-8bb8-aec200b04fd4,4,sunflower (micro),10,100.0,3,3.0,42.5,15.0,Farm
4,C,6,1,0.0,2022-07-04 14:57:55.951670,7ec47783-8c1b-497d-a5f1-aec500dc9f44,5,green mizuna (petite),5,0.0,4,3.0,0.0,100.0,Farm


In [39]:
cols_to_normal_sample = ['crop_yield', 'number_of_trays','over_production', 'waste_defect', 'waste_disease']

date_samples = pd.date_range(start=df['event_time'].min(),
                  end=pd.to_datetime('today'),
                  periods=10000)

parameter_samples = {}

for col in df.columns:
    if col == 'event_time':
        parameter_samples[col] = date_samples
        continue
    parameter_samples[col] = df[col].unique()

def build_row_dict(parameters, df):
    row = {}
    # one option is just shuffle existing data i.e.: 
    # row = {k:np.random.choice(v) for k,v in parameters.items()}

    # but lets make numbers for things a bit random to add something somewhat
    # interesting and varied? 

    for k,v in parameters.items():
        row[k] = np.random.choice(v) if k not in cols_to_normal_sample else np.floor(np.abs(np.random.normal(df[k].mean(), df[k].std())))

    return row 

def make_table_synth_data(parameter_samples, df, n=100):
    data = [build_row_dict(parameter_samples, df) for _ in range(n)]
    return pd.DataFrame(data)

synth_data = make_table_synth_data(parameter_samples, df)
synth_data.head()

,aisle,batch_id,column,crop_yield,event_time,growapp_id,id,name,number_of_trays,over_production,shelf,tray_size,waste_defect,waste_disease,zone
0,E,12,11,4621.0,2022-07-14 22:45:06.336048702,f28279e0-d0fd-463f-8455-aeca00d8400f,10,green mizuna (petite),5.0,65.0,4,3.0,37.0,28.0,Farm
1,C,4,1,153.0,2022-07-21 13:28:36.819881929,fa35712f-09ff-4fca-9075-aeca00df2c22,6,green mizuna (petite),7.0,20.0,3,3.0,5.0,25.0,Farm
2,E,1,2,1335.0,2022-07-23 12:50:32.964263907,f28279e0-d0fd-463f-8455-aeca00d8400f,5,red cabbage (micro),6.0,71.0,3,3.0,4.0,47.0,Farm
3,C,1,2,936.0,2022-07-19 21:27:13.269309922,7ec47783-8c1b-497d-a5f1-aec500dc9f44,11,peashoots (babyleaf),5.0,32.0,4,3.0,56.0,39.0,Farm
4,C,8,2,2467.0,2022-07-24 01:17:44.716510233,f241c061-babb-432a-8861-aeca00dfd920,6,sunflower (micro),6.0,20.0,1,3.0,0.0,16.0,Farm
